# Simple Auto Correction

In this notebook a simple auto-correction is implemented.
This version 0 does not care about the context or meaning of the words. 
The principle is very simple: First we find all the words that are 1 or 2 edits away from the wrongly spelled word. Then, based on their probability we sort them and give n toppest options.
The probability for each word is the number of times in occurs in our reference corpus that we read from a file.

Although very simple, it is worth to get this version first before looking into more advanced attention model based auto-correction that the probability of the words is calculated based on the context.

In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd


The corpus is read from a file

In [2]:
def process_data(file_name):
    words = [] 
    
    f = open(file_name, 'r')
    
    text = f.read()
    
    text = text.lower()
    
    words = re.findall(r'\w+', text)
    
    return words

In [3]:
word_l = process_data('./data/shakespeare.txt')
vocab = set(word_l) 

A dictionary is created that stores the number of times each unique word has appeared in the corpus. Also the created set of unique words will be used as our vocabulary, and we can check the existance of the word in it later.

In [4]:
def get_count(word_l):
    
    word_count_dict = {}  
    vocab_ = set(word_l)
    for word in word_l:
        word_count_dict[word] = word_count_dict.get(word, 0) + 1
            
    return word_count_dict

In [5]:
word_count_dict = get_count(word_l)

From the previous dictionary another dictionary is created that stores the probabilty of each word. The probability is simpley the number of the times it has shown up the corpus, divided by the number of the words in the corpus.

In [6]:
def get_probs(word_count_dict):
    probs = {}  
    
    M = 0
    for word in word_count_dict:
        M += word_count_dict[word]
    
    for word in word_count_dict:
        probs[word] = word_count_dict[word] / M
    
    return probs

In [7]:
probs = get_probs(word_count_dict)

There are four possible edit options: Delete a character, Switch two adjacent characters, Replace a character with a different one, and Inserting a character. All these edits can occur at different locations for the word.

Here, for each of these edits a function is defined that returns a list for the edit at different locations along the word's length. 

Later on, another function would be define to call all of these for a given word, and combine the result of all the edits for it.

In [8]:
def delete_letter(word):
    
    delete_l = []
    split_l = []
    
    for i in range(len(word)):
        delete_l.append(word[:i] + word[i+1:])
    
    return  delete_l

In [9]:
def switch_letter(word):
    
    switch_l = []
    split_l = []
    
    n = len(word)
    for i in range(n):
        a = word[:i] if i >= 0 else ""
        b = word[i] 
        c = word[i+1] if i + 1 < n else ""
        d = word[i+2:] if i + 2 < n else ""
        if len(c) > 0:
            switch_l.append(a + c + b + d)
    
    return switch_l

In [10]:
def replace_letter(word, verbose=False):
    
    replace_l = []
    
    n = len(word)
    for i in range(n):
        a = word[:i]
        b = word[i]
        c = word[i+1:] if i + 1 < n else ""
        for j in range(26):
            rep = chr(j + ord('a'))
            if rep != word[i]:
                replace_l.append(a + (rep) + c)
    
    replace_set = set(replace_l)
    replace_l = sorted(list(replace_set))
    
    return replace_l

In [11]:
def insert_letter(word):
    
    insert_l = []
    split_l = []
    
    for i in range(len(word) + 1):
        split_l.append((word[:i], word[i:]))
    
    for a, b in split_l:
        for i in range(26):
            insert_l.append(a + chr(i + ord('a')) + b)
    
    return insert_l

This function applies all the edits (Delete, Switch, Replace, and Insert) and creates a list of possible options for 1 single edit to the word.

In [12]:
def edit_one_letter(word, allow_switches = True):
 
    edit_one_set = set()
    
    edit_one_set.update(delete_letter(word))
    if allow_switches == True:
        edit_one_set.update(switch_letter(word))
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))
    
    return set(edit_one_set)

We also need to look into two edits. For this, first we create the list of one edit options, and for each of them we can apply again the one edit. 

Some of the previous edits may be cancelled and may get options that are zero or one edit away instead but we can check for them later.

In [13]:
def edit_two_letters(word, allow_switches = True):
    
    edit_two_set = set()
    
    edit_one_set = edit_one_letter(word, allow_switches)
    for edit in edit_one_set:
        edit_two_set.update(edit_one_letter(edit, allow_switches))
        
    return set(edit_two_set)

Now that we have all the one edit and two edit options, we can get the correction for any word.

The principle is as follows:
- if the word is already in the vocab, no correction is needed and we only return that word.
- if the word has one edit options in the vocabulary, we sort them based on their probability, and return the desired top n words.
- if the word has two edit options in the vocabulary, we sort them based on their probability, and return the desired top n words.
- if the word has no one edit or two edit options in the vocabulary, we return the word itself

In [14]:
def get_corrections(word, probs, vocab, n=2):
    
    suggestions = []
    n_best = []
    
    suggestions = []
    if word in vocab:
        suggestions.append(word)
    else:
        for edit_one in edit_one_letter(word, allow_switches = True):
            if edit_one in vocab:
                suggestions.append(edit_one)
    if len(suggestions) == 0:
        for edit_two in edit_two_letters(word, allow_switches = True):
            if edit_two in vocab:
                suggestions.append(edit_two)
    if len(suggestions) == 0:
        suggestions.append(word)

    suggestion_prob = []
    for suggestion in suggestions:
        suggestion_prob.append([suggestion, probs.get(suggestion, 0)])
    
    suggestion_prob = sorted(suggestion_prob, key = lambda x : -x[1])
    
    n_best = [suggestion_prob[i] for i in range(min(n, len(suggestion_prob)))]
    
    return n_best

# Test and Example of the simple auto-correct

Please change the wrong_word in the cell below and check what corrected options we get:

In [18]:
wrong_word = "thrgh"
options = get_corrections(wrong_word, probs, vocab, n=4)

print("For word " + wrong_word + ", the corrected options are:")
for i, option in enumerate(options):
    print("option {} is {} with probability of {}".format(i + 1, option[0], option[1]))


For word thrgh, the corrected options are:
option 1 is though with probability of 0.0011377625247136942
option 2 is high with probability of 0.0003730368933487522
option 3 is three with probability of 0.00031708135934643936
option 4 is through with probability of 0.00024247398067668894


# Getting the minimum edit distance between two words

Given two words of target and source, we can find minimum edit distance between them using bottom-up dynamic programming.

In [19]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    
    m = len(source) 
    n = len(target) 
    D = np.zeros((m+1, n+1), dtype=int) 
    
    for row in range(1, m + 1): 
        D[row,0] = row * ins_cost
        
    for col in range(1, n + 1): 
        D[0,col] = col * ins_cost
        
    for row in range(1, m + 1):
        
        for col in range(1, n + 1):
            
            r_cost = rep_cost
            
            if source[row - 1] == target[col - 1]: 
                r_cost = 0
                
            D[row,col] = min(D[row - 1, col] + del_cost, D[row, col - 1] + ins_cost, D[row - 1, col - 1] + r_cost)
            
    med = D[m, n]
    
    return med

# Example of minimum edit distance between two words:

In [20]:
source = "high"
target = "through"

print("The minimum distance between {} and {} is {}".format(source, target, min_edit_distance(source, target, 1, 1, 2)))

The minimum distance between high and through is 5
